# **The HandTracker class**

This notebook provides some documentation about ```HandTracker``` objects.

Run the following codeblocks to import the ```HandTracker``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: c:\Users\Brad\Documents\Programming\fivecarddraw


In [2]:
from fivecarddraw import HandTracker

## **HandTracker Functionality**

### **Initialisation**

The ```HandTracker``` class is designed to serve as a backend for ```Dealer``` objects. Upon initialisation:
* It creates a ```Deck``` object that helps with the *dealing and collecting of cards*.
* It loads about 150kb of data from [lookup tables](../lookup-tables/) that help with the *evaluating of cards*. 

More details about both can be found below. 

### **The Dealing and Collecting of Cards**

```HandTracker``` objects have the following attributes that help with the dealing and collecting of cards:

* ```HandTracker.DECK``` which is a ```Deck``` object.
* ```HandTracker.hands``` which is a ```dict``` for storing information about player hands.


In [4]:
tracker = HandTracker()
print(f"Deck of cards: {tracker.DECK}")

Deck of cards: [8♡, A♢, Q♠, 3♣, 4♠, 2♡, 4♣, K♣, 5♡, A♡, J♣, K♢, 5♠, 4♢, 5♣, J♠, 5♢, 9♠, 4♡, 9♣, Q♡, 7♡, J♢, 6♣, 8♢, K♠, A♠, 3♡, A♣, 10♣, 7♠, 8♣, 10♢, 6♢, 8♠, 2♢, Q♢, 6♡, 3♠, 2♣, Q♣, J♡, 2♠, 10♡, 7♣, 3♢, K♡, 7♢, 6♠, 9♢, 9♡, 10♠]


The intended way to deal cards to players is to use ```HandTracker.DealHands(names)```. This results in the ```HandTracker.hands``` being updated correctly. 

In [5]:
tracker = HandTracker()
tracker.DealHands(["Brad", "Scott", "Daniel"])
print(f"Hands: {tracker.hands}\n")
print(f"Amount of remaining cards in deck: {tracker.DECK.CountRemaining()}")

Hands: {'Brad': {'cards': [Q♡, 4♡, 10♣, 2♠, 4♣]}, 'Scott': {'cards': [3♡, 5♡, 6♡, 9♣, 10♢]}, 'Daniel': {'cards': [A♠, 3♢, A♡, Q♠, 6♢]}}

Amount of remaining cards in deck: 37


The intended way to collect cards from players is then to use ```HandTracker.CollectHands()```, which clears ```HandTracker.hands``` and prepares ```HandTracker.DECK``` for a new round.

In [6]:
tracker = HandTracker()
tracker.DealHands(["Brad", "Scott", "Daniel"])
print(f"Amount of remaining cards in deck before collecting hands: {tracker.DECK.CountRemaining()}")
tracker.CollectHands()
print(f"Amount of remaining cards in deck after collecting hands: {tracker.DECK.CountRemaining()}")

Amount of remaining cards in deck before collecting hands: 37
Amount of remaining cards in deck after collecting hands: 52


#### **Demo of Hands**

Alternatively, the ```HandTracker.DemoDeal()``` method can be used to create a random hand without the implict tracking of ```Card``` objects. This method is purely for demonstration and is never called in [fivecarddraw.py](../fivecarddraw.py).

In [26]:
tracker = HandTracker()

In [35]:
print(f"Random hand: {tracker.DemoDeal()}")

Random hand: [Q♡, J♢, 8♡, A♡, 10♡]


### **The Evaluating of Hands**

```HandTracker``` objects have specific attributes that were designed to optimise ```HandTracker.EvaluateHands()``` to be as quick as possible. Essentially, data about every possible five card draw hand is loaded by the ```HandTracker``` object, from [lookup-tables](../lookup-tables/) upon initialisation, which can then simply be read to retrieve the evaluation of a hand. More specifically, the following attributes help with the evaluating of hands:

* ```HandTracker.DV``` and ```HandTracker.DP``` which are ```dict``` ciphers used during ```HandTracker.LoadLookupTables()```.
* ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` and ```HandTracker.DUPE_RANKS``` which are ```dict``` memos that store information loaded from [lookup-tables](../lookup-tables/) by ```HandTracker.LoadLookupTables()```.
* ```HandTracker.CLASSES``` and ```HandTracker.BOUNDARIES``` which are used to convert between the numerical and categorical evaluations of hands.


#### **looking at the lookup-tables**

[lookup-tables](../lookup-tables/) contains three files which are tables of data that contain both numerical and categorical evaluations of every possible five card draw hand. The data was scraped from [here](http://suffe.cool/poker/7462.html) and divided into three seperate tables to simplify the loading process used in ```HandTracker.LoadLookupTables()```. Each file is formatted in the same way, and can be previewed below. 

In [3]:
flush_lookup = open("lookup-tables/flush lookup.txt", "r")
unique5_lookup = open("lookup-tables/unique five lookup.txt", "r")
dupe_lookup = open("lookup-tables/dupe lookup.txt", "r")

In [20]:
print(flush_lookup.readline())
print(unique5_lookup.readline())
print(dupe_lookup.readline())

FF  AKQJ3  329

HC  AKQJ3  6192

4K  KKKK9  27



In [25]:
flush_lookup.close()
unique5_lookup.close()
dupe_lookup.close()

Each line can be interpreted as follows:

* The first two letters encode a categorical evaluation; either SF, 4K, FH, FF, SS, 3K, 2P, 1P or HC which stand for straight flush, four of a kind, full house, flush, straight, three of a kind, two pair, pair or high card respectively.
* The next five characters give the cards in a hand; either A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2.
* The remaining numbers give a numerical evaluation; 1 being the best and 7462 being the worst, which were determined through combinatorial analysis by [Kevin Suffecool](http://suffe.cool/poker/evaluator.html).

This information is stored by the ```HandTracker``` object as outlined below.

#### **Loading the data from the lookup-tables**

```HandTracker``` objects have the following attributes that are initialised before ```HandTracker.LoadLookupTables()``` is called:

* ```HandTracker.DV = {char : 2 ** i for i, char in enumerate("23456789TJQKA")}``` which is a cipher that allows hands which are classed as either a straight flush, flush, straight or having only a high card to be encoded as a unique sum of powers of two.
* ```HandTracker.DP = {char : p for p, char in zip(Card(0,0).PRIMES, "23456789TJQKA")}``` which is a cipher that allows any hand to be encoded as a unique product of primes instead.

In [32]:
tracker = HandTracker()
print(f"Sum cipher: {tracker.DV}")
print(f"product cipher: {tracker.DP}\n")

Sum cipher: {'2': 1, '3': 2, '4': 4, '5': 8, '6': 16, '7': 32, '8': 64, '9': 128, 'T': 256, 'J': 512, 'Q': 1024, 'K': 2048, 'A': 4096}
product cipher: {'2': 2, '3': 3, '4': 5, '5': 7, '6': 11, '7': 13, '8': 17, '9': 19, 'T': 23, 'J': 29, 'Q': 31, 'K': 37, 'A': 41}



The ```HandTracker.LoadLookupTables()``` method then creates the following attributes: 

* ```HandTracker.FLUSH_RANKS``` which is a ```dict``` whose keys are unique sums representing hands, and whose values are the numerical evaluation of them, assuming each card has the same suit. Data comes from [flush lookup.txt](../lookup-tables/flush_lookup.txt).
* ```HandTracker.UNIQUE5_RANKS``` which is a ```dict``` whose keys are unique sums representing hands, and whose values are the numerical evaluation of them, assuming there is more than one suit present in the hand. Data comes from [unique five lookup.txt](../lookup-tables/unique_five_lookup.txt).
* ```HandTracker.DUPE_RANKS``` which is a ```dict``` whose keys are unique products representing hands, and whose values are the numerical evaluation of them. Data comes from [dupe lookup.txt](../lookup-tables/dupe_lookup.txt).

In [35]:
tracker = HandTracker()
print(f"Amount of hands in FLUSH_RANKS: {len(tracker.FLUSH_RANKS)}")
print(f"Amount of hands in UNIQUE5_RANKS: {len(tracker.UNIQUE5_RANKS)}")
print(f"Amount of hands in DUPE_RANKS: {len(tracker.DUPE_RANKS)}")

Amount of hands in FLUSH_RANKS: 1287
Amount of hands in UNIQUE5_RANKS: 1287
Amount of hands in DUPE_RANKS: 4888


#### **Getting a Numerical Evaluation**

In order to get a numerical evaluation of a hand, it just needs to be located. The first step is to find which ```dict``` attribute of ```HandTracker``` it's located in. It will either be located in  ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` or ```HandTracker.DUPE_RANKS```. The first check to do is to check whether the hand contains a flush or not. This is done using ```HandTracker.CheckFlush(cards)```.

##### **Finding where the numerical evaluation is located**

In [21]:
tracker = HandTracker()

In [28]:
hand = tracker.DemoDeal()
print(f"Random hand: {hand}")
print(f"Has a flush: {tracker.CheckFlush(hand)}")


Random hand: [J♠, 6♢, 8♣, A♡, 8♠]
Has a flush: False


If the hand doesn't contain a flush, then it needs to be determined whether the hand contains duplicates or not. This is done using ```HandTracker.CheckUnique5(cards)```.

In [29]:
tracker = HandTracker()

In [34]:
hand = tracker.DemoDeal()
print(f"Random hand: {hand}")
print(f"Has five unique cards: {tracker.CheckUnique5(hand)}")
print(f"Has duplicate cards: {not tracker.CheckUnique5(hand)}")

Random hand: [A♢, J♡, 4♠, 6♠, K♡]
Has five unique cards: True
Has duplicate cards: False


Once the above details have been found, then the location of the hand's evaluation can be established. 

* If the hand has a flush, then it's evaluation is in ```HandTracker.FLUSH_RANKS```.
* If the hand doesn't have a flush, but it does have 5 unique-calued cards, then it's evaluation is in ```HandTracker.UNIQUE5_RANKS```.
* Otherwise the hand's evaluation is in ```HandTracker.DUPE_RANKS```.

##### **Getting the key to obtain the numerical evaluation**

If the hand's evaluation is in ```HandTracker.DUPE_RANKS```, then it's key can be found using ```HandTracker.ExtractProduct(cards)```.

In [37]:
tracker = HandTracker()
hand = tracker.DemoDeal()
print(f"Random hand: {hand}")
print(f"Unique product encoding: {tracker.ExtractProduct(hand)}")

Random hand: [7♢, Q♣, 5♢, 2♠, 9♣]
Unique product encoding: 107198


Otherwise it's key can be found using ```HandTracker.ExtractSum(card)```.

In [44]:
tracker = HandTracker()
hand = tracker.DemoDeal()
print(f"Random hand: {hand}")
print(f"Sum encoding: {tracker.ExtractSum(hand)}")

Random hand: [8♠, A♣, 10♢, 8♣, 6♡]
Sum encoding: 4432


These keys can then be used with the respective ```dict``` attribute of ```HandTracker``` to obtain the numerical evaluation of the hand.

#### **Getting a Categorical Evaluation**

```HandTracker``` objects have the following attributes that help with categorical evaluation:

* ```HandTracker.CLASSES = ("High card", "pair", "two pair", "three of a kind", "straight", "flush", "full house","four of a kind", "straight flush", "royal flush")```
* ```HandTracker.BOUNDARIES = (6186, 3326, 2468, 1610, 1600, 323, 167, 11, 2, 1)```

These attributes compose a cipher, from which a categorical evaluation can be determined from the hand's numerical evaluation. 

### **Demo of Evaluations**

The intended method to use to evaluate hands is ```HandTracker.EvaluateHands()```.

In [3]:
tracker = HandTracker()

In [10]:
tracker.DealHands(["Brad", "Scott", "Daniel"])
tracker.EvaluateHands()
print(f"Player Hands: {tracker.hands}")
tracker.CollectHands()

Player Hands: {'Brad': {'cards': [K♡, 7♣, 6♡, 5♢, 9♢], 'rank_n': 6953, 'rank_c': 'High card'}, 'Scott': {'cards': [10♢, 5♠, 4♢, 8♠, 8♡], 'rank_n': 4825, 'rank_c': 'pair'}, 'Daniel': {'cards': [K♠, 7♡, 9♡, 3♢, 5♣], 'rank_n': 6958, 'rank_c': 'High card'}}


Alternatively, hands can be evaluated using ```HandTracker.DemoEvaluate(hand)``` which doesn't implicitly track who has the hand.

In [3]:
tracker = HandTracker()

In [39]:
hand = tracker.DemoDeal()
print(f"Player Hand: {hand}")
print(f"Evaluation: {tracker.DemoEvaluate(hand)['rank_c']}")

Player Hand: [7♢, 2♠, 10♣, 9♣, 3♢]
Evaluation: High card


### **The Switching of Cards**

The intended way to switch cards during the switching phase of a round of five card draw poker; is to first approve the selection of cards to be discarded using ```HandTracker.ApproveDiscards(name, discards)```, and once that returns ```True```, proceed to edit the hand using ```HandTracker.EditHand(name, discards)```. 

In [20]:
tracker = HandTracker()
tracker.DealHands(["Brad", "Scott", "Daniel"])
tracker.EvaluateHands()
print(f"Player Hands: {tracker.hands}")

Player Hands: {'Brad': {'cards': [J♠, 3♢, Q♡, J♢, A♠], 'rank_n': 4003, 'rank_c': 'pair'}, 'Scott': {'cards': [10♢, 5♣, 3♠, 8♠, 6♠], 'rank_n': 7387, 'rank_c': 'High card'}, 'Daniel': {'cards': [10♡, J♡, 5♢, 2♡, 9♡], 'rank_n': 7233, 'rank_c': 'High card'}}


In [23]:
player = "Brad"
discards = tracker.hands[player]["cards"][:4]
remaining = tracker.hands[player]["cards"][4]
approval = tracker.ApproveDiscards(player, discards)

print(f"Four cards to try and switch: {discards}")
print(f"Remaining card: {remaining}")
print(f"Approval : {approval}")

Four cards to try and switch: [J♠, 3♢, Q♡, J♢]
Remaining card: A♠
Aprroval : True


In [24]:
tracker.EditHand(player, discards)
print(f"Player Hands: {tracker.hands}")


Player Hands: {'Brad': {'cards': [A♠, 7♡, A♡, Q♢, 4♡], 'rank_n': 4003, 'rank_c': 'pair'}, 'Scott': {'cards': [10♢, 5♣, 3♠, 8♠, 6♠], 'rank_n': 7387, 'rank_c': 'High card'}, 'Daniel': {'cards': [10♡, J♡, 5♢, 2♡, 9♡], 'rank_n': 7233, 'rank_c': 'High card'}}
